In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.linear_model import LinearRegression
import sklearn.metrics as met
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [2]:
#define dataset
data= pd.read_csv('...//feature.csv')

In [3]:
#split
x= data.iloc[:,:(data.shape[1]-1)]
y= data.iloc[:,(data.shape[1]-1)]

x_train, x_test, y_train, y_test = train_test_split(x, y ,test_size=0.25)


'''Two methods to reduce the dimension, first : remove low variance features and high corr features and fit PCA second: just fit PCA '''

In [4]:
#find and remove low variance feature
high_var=set()
for i in range(len(x_train.columns)):
        if x_train.iloc[:,i].var() == 0 :
            colname = x_train.columns[i]
            high_var.add(colname)




In [11]:
x_train.drop(labels=high_var, axis=1, inplace=True)
x_test.drop(labels=high_var, axis=1, inplace=True)

KeyError: "['197' '236' '112' '493' '434' '248' '17' '25' '153' '494' '140' '374'\n '192' '372' '133' '385' '150' '134' '352' '491' '166' '46' '474' '254'\n '464' '255' '158' '269' '347' '101' '458' '466' '507' '73' '499' '152'\n '502' '85' '470' '3' '435' '284' '465' '291' '504' '174' '412' '214'\n '258' '337' '36' '137' '451' '53' '33' '8' '424' '126' '349' '240' '320'\n '15' '93' '429' '116' '143' '345' '229' '292' '294' '90' '290' '66' '122'\n '223' '217' '393' '135' '26' '24' '246' '287' '418' '422' '201' '259'\n '376' '301' '187' '16' '48' '128' '289' '119' '488' '225' '203' '335'\n '251' '170' '245' '123' '196' '163' '382' '204' '276' '339' '331' '74'\n '76' '490' '379' '111' '438' '498' '221' '206' '237' '234' '146' '72'\n '57' '463' '63' '288' '468' '262' '179' '186' '78' '1' '222' '70' '455'\n '497' '439' '456' '275' '274' '344' '324' '185' '277' '131' '29' '52'\n '65' '40' '2' '373' '19' '67' '270' '340' '114' '107' '10' '127' '230'\n '100' '449' '162' '411' '232' '191' '501

In [9]:
#find and remove high correlation
corr_matrix2 = x_train.corr()
correlated_var=set()

for i in range(len(corr_matrix2 .columns)):
    for j in range(i):
        if abs(corr_matrix2.iloc[i, j]) > 0.85:
            colname = corr_matrix2.columns[i]
            correlated_var.add(colname)

In [10]:
x_train.drop(labels=correlated_var, axis=1, inplace=True)
x_test.drop(labels=correlated_var, axis=1, inplace=True)


In [12]:
#scale
scaler=MinMaxScaler()
x_train_scaled=scaler.fit_transform(x_train)
x_test_scaled=scaler.transform(x_test)

In [13]:
#fitting pca
pca = PCA(n_components=30)
pca.fit(x_train_scaled)

x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)


In [6]:
#RF

rf_model = RandomForestClassifier()
#here you can put any parameter you want at every run, like random_state or verbosity
params_to_test ={"n_estimators":range(2, 60),"max_depth":range(2, 20) }

#here you specify the CV parameters, number of folds, numberof cores to use...
grid_search = GridSearchCV(estimator=rf_model, param_grid=params_to_test, cv=10 , n_jobs=4)

grid_search.fit(x_train_pca, y_train)
print(grid_search.best_params_ ) #max_depth= 6, n_estimators= 56=8 
print(grid_search.best_score_)


{'max_depth': 10, 'n_estimators': 38}
0.95


In [7]:
rf_model= RandomForestClassifier(max_depth= 10, n_estimators= 38)
rf_model.fit(x_train_pca, y_train)
print("Train Acc is : ",accuracy_score(y_train,rf_model.predict(x_train_pca)))
print("Test Acc is : ",accuracy_score(y_test,rf_model.predict(x_test_pca)))   

Train Acc is :  1.0
Test Acc is :  0.8461538461538461


In [14]:
#KNN
knn_model = KNeighborsClassifier()
param_knn = {'n_neighbors' : np.arange(1 , 34)}
grid_knn = GridSearchCV(knn_model , param_knn , cv = 10) 
grid_knn.fit(x_train_pca, y_train)
print(grid_knn.best_params_ ) # #'n_neighbors': 2
print(grid_knn.best_score_)


{'n_neighbors': 1}
0.8666666666666666


In [15]:
knn_model=KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train_pca, y_train)

print("Train Acc is : ",accuracy_score(y_train,knn_model.predict(x_train_pca)))
print("Test Acc is : ",accuracy_score(y_test,knn_model.predict(x_test_pca)))

Train Acc is :  1.0
Test Acc is :  0.9230769230769231


In [10]:
#SVM
svm_model = svm.SVC()
svm_param = {"C": [10, 50, 100, 200, 300, 400, 500],
             "kernel" :['linear', 'poly', 'rbf', 'sigmoid'],
              "gamma": [0.001,0.002,0.01,0.02,0.01]}
grid_svm = GridSearchCV(estimator=svm_model , param_grid = svm_param , cv = 10)
grid_svm.fit(x_train_pca , y_train)
grid_svm.best_params_  #{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}  


{'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}

In [11]:
svm_model = svm.SVC(kernel='rbf', C=10, gamma=0.001)
svm_model.fit(x_train_pca,y_train)

print("Train Acc is : ",accuracy_score(y_train,svm_model.predict(x_train_pca))) # 1.00  #97
print("Test Acc is : ",accuracy_score(y_test,svm_model.predict(x_test_pca)))  #92    #92


Train Acc is :  0.972972972972973
Test Acc is :  1.0


In [13]:
#DT
tree = DecisionTreeClassifier()
param_df = {'max_depth' : [None , 3], 'max_features': np.arange(1,30) ,
            'min_samples_leaf': np.arange(1,10)}
grid_tree = GridSearchCV(tree , param_df , cv=10)
grid_tree.fit(x_train_pca, y_train)
print(grid_tree.best_params_ ) #'max_depth': 3, 'max_features': 13, 'min_samples_leaf': 1
print(grid_tree.best_score_)

{'max_depth': 3, 'max_features': 17, 'min_samples_leaf': 4}
0.925


In [14]:

tree_model=DecisionTreeClassifier(max_depth=3, max_features= 17, min_samples_leaf= 4)
tree_model.fit(x_train_pca,y_train)
print("Train Acc is : ",accuracy_score(y_train,tree_model.predict(x_train_pca))) # 83  #86
print("Test Acc is : ",accuracy_score(y_test,tree_model.predict(x_test_pca)))  #92   #77

Train Acc is :  0.918918918918919
Test Acc is :  0.6153846153846154


In [15]:
#linear regression
lireg_model = LinearRegression()
lireg_model.fit(x_train_pca , y_train)
met.mean_squared_error(y_train,lireg_model.predict(x_train_pca))  #0.044  #0.02
met.mean_squared_error(y_test,lireg_model.predict(x_test_pca))  #0.46     #0.31


0.22200564530412753